In [0]:
dbutils.widgets.text("url","https://jsonplaceholder.typicode.com/users")
dbutils.widgets.text("output_path","/Volumes/auto_loader/auto_loader_sch/auto_load_vol/json_data_source/json_url_data")
output_path=dbutils.widgets.get("output_path")
json_url=dbutils.widgets.get("url")
print(output_path)
print(json_url)


In [0]:
import json
import requests
from datetime import datetime

def get_url_response(url,output_path):

    try:
        response = requests.get(url)

        if response.status_code == 200:
            print("sucessfully retrived response from the url")
        else:
            print("Error while retriving response..retrying")
            response = requests.get(url)
            if response.status_code==200:
                print("sucessfully retrived response from the url")
            else:
                print("The reponse from the url is {response.status_code}")
                return
        
        try:
            data = response.json()
            print("Data received successfully:")
        except json.JSONDecodeError:
            print("Error: The response is not valid JSON.")
            return
        ts = datetime.now().strftime("%Y%m%d%H%M%S")
    
        final_output_path = f"{output_path}_{ts}.json"
       
        
        try:
            dbutils.fs.put(final_output_path,json.dumps(data),overwrite=True)
            
            print(f"Successfully wrote data to {final_output_path}")
           
        except NameError:
            print("Error: 'dbutils' is not defined. This code must run in a Databricks notebook.")
            return
    except Exception as e:
        print(f"Error: {e}")
        return


In [0]:
get_url_response(json_url,output_path)

In [0]:
from pyspark.sql.types import StructType, StructField
from pyspark.sql.types import StringType, LongType
schemas = StructType([

    StructField("address",StringType(),True),
    StructField("company", StringType(), True),
    StructField("email", StringType(), True),
    StructField("id", LongType(), True),
    StructField("name", StringType(), True),
    StructField("phone", StringType(), True),
    StructField("username", StringType(), True),
    StructField("website", StringType(), True)

])
df1=spark.readStream.format("cloudfiles")\
.option("cloudFiles.format","json")\
.option("cloudFiles.schemaLocation","/Volumes/auto_loader/auto_loader_sch/auto_load_vol/json_data_metadata/json_schema")\
.option("cloudFiles.maxFilesPerTrigger","1")\
.option("checkpointLocation","/Volumes/auto_loader/auto_loader_sch/auto_load_vol/json_data_metadata/chkp")\
.load("/Volumes/auto_loader/auto_loader_sch/auto_load_vol/json_data_source")

In [0]:
from pyspark.sql.functions import from_json,col
schema1=StructType([
    StructField("name",StringType(),True),
    StructField("catchPhrase",StringType(),True),
    StructField("bs",StringType(),True)
])

schema2=StructType([
    StructField("lat",StringType(),True),
    StructField("lng",StringType(),True)
])
schema3=StructType([
    StructField("street",StringType(),True),
    StructField("suite",StringType(),True),
    StructField("city",StringType(),True),
    StructField("zipcode",StringType(),True),
    StructField("geo",schema2)
])


df2=df1.withColumn("company",from_json(col("company"),schema1)).withColumn("address",from_json(col("address"),schema2))




In [0]:

df2.writeStream.format("delta").trigger(availableNow=True)\
        .option(
            "checkpointLocation",
            "/Volumes/auto_loader/auto_loader_sch/auto_load_vol/json_data_metadata/chkp")\
        .start(
            "/Volumes/auto_loader/auto_loader_sch/auto_load_vol/json_data/")

In [0]:
df2.writeStream.format("delta").trigger(availableNow=True).option(
            "checkpointLocation",
            "/Volumes/auto_loader/auto_loader_sch/auto_load_vol/json_data_metadata/chkp"
        )\
        .toTable("auto_loader.auto_loader_sch.api_stream_data")